***

# Geothermal Well Test Analysis with Python
### Bonus material: Combine PTS and pump flow rate data
#### Irene Wallis and Katie McLean 
#### Software Underground, Transform 2021

***

In [13]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime
import openpyxl


#from utilities import* # functions in the utilities.py file

In [14]:
def timedelta_seconds(dataframe_col, test_start):
    '''
    Make a float in seconds since the start of the test

    args:   dataframe_col: dataframe column containing datetime objects
            test_start: test start time formatted '2020-12-11 09:00:00'

    returns: float in seconds since the start of the test
    '''
    test_start_datetime = pd.to_datetime(test_start)
    list = []
    for datetime in dataframe_col:
        time_delta = datetime - test_start_datetime
        seconds = time_delta.total_seconds()
        list.append(seconds)
    return list



def read_pts(filename):
    '''
    Read PTS-2.xlsx in as a Pandas dataframe and munge for analysis

    args: filename is r'PTS-2.xlsx'

    returns: Pandas dataframe with datetime (local) and key coloumns of PTS data with the correct dtype
    '''
    df = pd.read_excel(filename)

    dict = {
        'DEPTH':'depth_m',
        'SPEED': 'speed_mps',
        'Cable Weight': 'cweight_kg',
        'WHP': 'whp_barg',
        'Temperature': 'temp_degC',
        'Pressure': 'pressure_bara',
        'Frequency': 'frequency_hz'
    }
    df.rename(columns=dict, inplace=True)

    df.drop(0, inplace=True)
    df.reset_index(drop=True, inplace=True)

    list = []
    for date in df.Timestamp:
        newdate = openpyxl.utils.datetime.from_excel(date)
        list.append(newdate)
    df['datetime'] = list

    df.drop(columns = ['Date', 'Time', 'Timestamp','Reed 0',
       'Reed 1', 'Reed 2', 'Reed 3', 'Battery Voltage', 
       'PRT Ref Voltage','SGS Voltage', 'Internal Temp 1', 
       'Internal Temp 2', 'Internal Temp 3','Cal Temp', 
       'Error Code 1', 'Error Code 2', 'Error Code 3',
       'Records Saved', 'Bad Pages',], inplace = True)
    
    df[
        ['depth_m', 'speed_mps','cweight_kg','whp_barg','temp_degC','pressure_bara','frequency_hz']
    ] = df[
        ['depth_m','speed_mps','cweight_kg','whp_barg','temp_degC','pressure_bara','frequency_hz']
        ].apply(pd.to_numeric)
    
    df['timedelta_sec'] = timedelta_seconds(df.datetime, '2020-12-11 09:26:44.448')

    return df



def read_flowrate(filename):
    ''' 
    Read PTS-2-injection-rate.xlsx in as a pandas dataframe and munge for analysis

    args: filename is r'PTS-2-injection-rate.xlsx'

    returns: pandas dataframe with local NZ datetime and flowrate in t/hr
    '''
    df = pd.read_excel(filename, header=1) 
    df.columns = ['raw_datetime','flow_Lpm']

    list = []
    for date in df['raw_datetime']:
        newdate = datetime.fromisoformat(date)
        list.append(newdate)
    df['ISO_datetime'] = list 

    list = []
    for date in df.ISO_datetime:
        newdate = pd.to_datetime(datetime.strftime(date,'%Y-%m-%d %H:%M:%S'))
        list.append(newdate)
    df['datetime'] = list

    df['flow_tph'] = df.flow_Lpm * 0.060

    df['timedelta_sec'] = timedelta_seconds(df.datetime, '2020-12-11 09:26:44.448')

    df.drop(columns = ['raw_datetime', 'flow_Lpm', 'ISO_datetime'], inplace = True)

    return df

In [15]:
flowrate = read_flowrate(r'Data-FlowRate.xlsx')
pts = read_pts(r'Data-PTS.xlsx')

In [16]:
flowrate.columns

Index(['datetime', 'flow_tph', 'timedelta_sec'], dtype='object')

In [17]:
flowrate = flowrate.set_index('timestamp')
flowrate.head(3)

KeyError: "None of ['timestamp'] are in the columns"

In [ ]:
pts = pts.set_index('timestamp')
pts.head(3)

In [ ]:
combined = pts.join(flowrate, how = 'outer',  lsuffix = '_pts', rsuffix = '_fr', )

In [ ]:
combined.columns

In [ ]:
combined.head(3)

In [ ]:
combined.drop(columns = ['datetime_fr'], inplace = True)

In [ ]:
combined.columns

In [ ]:
combined.columns = ['depth_m', 'speed_mps', 'cweight_kg', 'whp_barg', 'temp_degC',
       'pressure_bara', 'frequency_hz', 'datetime', 'flow_tph']

In [ ]:
combined.columns

In [ ]:
combined.describe()

In [ ]:
combined['flow_tph'].isna().sum()

In [ ]:
combined['new_flow_tph'] = combined['flow_tph'].interpolate(method='linear')

In [ ]:
combined['new_flow_tph'].isna().sum()

In [ ]:
combined.tail()

In [ ]:
fig, (ax) = plt.subplots(1, 1, figsize=(24,8))

ax.scatter(combined.datetime, combined.new_flow_tph, 
    c='k', s = 5, linewidths = 0)

ax.scatter(combined.datetime, combined.flow_tph, 
    c='r', s = 30, linewidths = 0)

ax.scatter(pts.datetime, pts.cweight_kg, 
    c='b', s = 5, linewidths = 0, label = 'Raw cabel weight')

ax.scatter(remaining_nan.datetime, remaining_nan.cweight_kg, 
    c='g', s = 30, linewidths = 0, label = 'NAN cabel weight')

ax.set_ylim(0,150)

ax.set_xlabel('Time [hh:mm]')
ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))


start_time = pd.to_datetime('2020-12-11 09:25:00')
end_time = pd.to_datetime('2020-12-11 09:30:00')

#ax.set_xlim(start_time,end_time);

As the plot and the remaining_nan.describe() show that I have filled all the PTS values with flow rates, I will now drop the flow values that have no pts value

In [ ]:
nan_trimmed = combined.dropna()

nan_trimmed = combined[combined['depth_m'].notna()]


In [ ]:
nan_trimmed.shape 

In [ ]:
combined.shape

In [ ]:
nan_trimmed.reset_index(inplace=True)

In [ ]:
nan_trimmed.head(2)

In [ ]:
fig, (ax) = plt.subplots(1, 1, figsize=(24,8))

ax.scatter(combined.datetime, combined.new_flow_tph, 
    c='k', s = 5, linewidths = 0)

ax.scatter(combined.datetime, combined.flow_tph, 
    c='r', s = 30, linewidths = 0)

ax.scatter(pts.datetime, pts.cweight_kg, 
    c='b', s = 5, linewidths = 0, label = 'Raw cabel weight')

ax.scatter(nan_trimmed.datetime, nan_trimmed.cweight_kg, 
    c='g', s = 30, linewidths = 0, label = 'NAN cabel weight')

ax.set_ylim(0,150)

ax.set_xlabel('Time [hh:mm]')
ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))


start_time = pd.to_datetime('2020-12-11 09:25:00')
end_time = pd.to_datetime('2020-12-11 09:30:00')

#ax.set_xlim(start_time,end_time);

In [ ]:
# turning the method into a function

flowrate_f = read_flowrate(r'Data-FlowRate.xlsx')
pts_f = read_pts(r'Data-PTS.xlsx')

In [ ]:
def append_flowrate_to_pts(flowrate_df, pts_df):
    '''
    Add surface flowrate to pts data

    Note that the flowrate data is recorded at a courser time resolution than the pts data
    The function makes a linear interpolation to fill the data gaps
    Refer to bonus-combine-data.ipynb to review this method and adapt it for your own data

    Args:   flowrate and pts dataframes generated by the read_flowrate and read_pts functions

    Returns: pts dataframe with flowrate tph added
    '''
    flowrate_df = flowrate_df.set_index('timestamp')
    pts_df = pts_df.set_index('timestamp')
    combined_df = pts_df.join(flowrate_df, how = 'outer',  lsuffix = '_pts', rsuffix = '_fr')
    combined_df.drop(columns = ['datetime_fr'], inplace = True)
    combined_df.columns = ['depth_m', 'speed_mps', 'cweight_kg', 'whp_barg', 'temp_degC',
       'pressure_bara', 'frequency_hz', 'datetime', 'flow_tph']
    combined_df['interpolated_flow_tph'] = combined_df['flow_tph'].interpolate(method='linear')
    trimmed_df = combined_df[combined_df['depth_m'].notna()]
    trimmed_df.reset_index(inplace=True)
    return trimmed_df

mudged_df = append_flowrate_to_pts(flowrate_f, pts_f)

In [ ]:
mudged_df.shape

In [ ]:
mudged_df.head()

***

<p><center>© 2021 <a href="https://www.cubicearth.nz/">Irene Wallis</a> and <a href="https://www.linkedin.com/in/katie-mclean-25994315/">Katie McLean</a> <a href="https://creativecommons.org/licenses/by/4.0/"</a></center></p>

<p><center>Licensed under the Apache License, Version 2.0</center></p>

***